#### PREPROCESADO DE DATOS

In [3]:
from comet_ml import Experiment

experiment = Experiment(api_key="ummagUWZ5eIZzmhPtFkA8oopu")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
[codecarbon INFO @ 12:26:44] [setup] RAM Tracking...
[codecarbon INFO @ 12:26:44] [setup] GPU Tracking...
[codecarbon INFO @ 12:26:44] No GPU found.
[codecarbon INFO @ 12:26:44] [setup] CPU Tracking...
[codecarbon WARNING @ 12:26:44] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 12:26:45] We saw that you have a Intel(R) Xeon(R) CPU E5-2630 v3 @ 2.40GHz but we don't know it. Please contact us.
[codecarbon INFO @ 12:26:45] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU E5-2630 v3 @ 2.40GHz
[codecarbon INFO @ 12:26:45] >>> Tracker's metadata:
[codecarbon INFO @ 12:26:45]   Platform system: Linux-5.4.0-122-generic-x86_64-with-glibc2.29
[codecarbon INFO @ 12:26:45]   Python version: 3.8.10
[codecarbon INFO @ 12:26:45]   Available RAM : 31.360 GB
[codecarbon INFO @ 12:2

In [16]:
%%capture

from codecarbon import EmissionsTracker
from comet_ml import Experiment

emissions_tracker = EmissionsTracker(project_name="Notebook TFM")

emissions_tracker.start()

### 0. Importing the libraries

In [2]:
import random
import json
import shutil
import re
import os
from tqdm import tqdm
from zipfile import ZipFile

import csv
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle

import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv

### 1.  Extracting Unique Track, Artist, Album URIs and the Playlist data from Spotify Million Playlist Dataset 

This code extracts the track_uri, artist_uri, and album_uri from the JSON files in the Spotify Million Playlist Dataset. It also formats the CSV files to make them more compact and selects only the unique tracks, removing any repeated tracks present in the playlists.

In [28]:
def read_mpd_data(folderpath):

    #Creating empty Dataframe with required Columns
    tracks_df = pd.DataFrame(columns=['pid','track_uri','artist_uri', 'album_uri'])

    # Fetching the data from the mpd data folder
    json_files = [f for f in os.listdir(folderpath) if os.path.splitext(f)[1] == '.json']
    json_files.sort()

    # Fetching pid and track_uri for all the playlists
    for index, js in tqdm(enumerate(json_files), total=len(json_files)):
        with open(os.path.join(folderpath, js)) as json_file:
            j = json.load(json_file)

            # extracting tracks from playlists in each slice
            tracks = pd.json_normalize(j['playlists'], record_path='tracks',
                                meta=['pid'])
            # append tracks to tracks_df
            tracks = tracks[['pid', 'track_uri', 'artist_uri', 'album_uri']]
            tracks_df = tracks_df.append(tracks)

    # Writing the tracks DataFrame to a CSV file
    tracks_df.to_csv('working/tracks_df.csv', index=False)
    print("CSV file successfully written!")

In [29]:
data_dir = 'spotify-million-playlist/data'
read_mpd_data(data_dir)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [2:20:15<00:00,  8.42s/it]


CSV file successfully written!


In [11]:
tracks_df = pd.read_csv('working/tracks_df.csv')

for col in tracks_df.columns:
    tracks_df[[col]].to_csv(f'working/{col}.csv', index=False)
    print(f"{col}.csv file successfully written!")

pid.csv file successfully written!
track_uri.csv file successfully written!
artist_uri.csv file successfully written!
album_uri.csv file successfully written!


In [32]:
track_uri = list(set(pd.read_csv('working/track_uri.csv')['track_uri']))
with open('working/unique_tracks.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(track_uri)
    
album_uri = list(set(pd.read_csv('working/album_uri.csv')['album_uri']))
with open('working/unique_album.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(album_uri)
    
artist_uri = list(set(pd.read_csv('working/artist_uri.csv')['artist_uri']))
with open('working/unique_artist.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(artist_uri)

 This code extracts the Playlist data from the JSON files.
 
We are reducing the size of the dataset because the subsequent processing tasks are computationally intensive and may be difficult to handle with the original dataset size.

We randomly select only 20 files (equal to 20.000 playlists) of a total of 1.000 json files.

In [29]:
data_dir = 'spotify-million-playlist/data'
num_files_to_select = 20

files = os.listdir(data_dir)
selected_files = random.sample(files, num_files_to_select)

playlists = []

for file in tqdm(selected_files, desc="Processing playlists"):
    file_path = os.path.join(data_dir, file)
    data = json.load(open(file_path))
    playlists.append(pd.DataFrame(data['playlists']))

playlists_data = pd.concat(playlists)
playlists_data.to_csv('working/playlists_data.csv', index=False)

Processing playlists: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:07<00:00,  2.53it/s]


A sample of the Playlist data.

In [30]:
playlists_data[playlists_data['description'].notna()].head(3)

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
19,Throwback,false,64019,1507766400,154,138,1,"[{'pos': 0, 'artist_name': 'Plain White T's', ...",131,34025164,118,A nostalgic collection of rock music.
71,Old But Good,false,64071,1477353600,81,60,2,"[{'pos': 0, 'artist_name': 'Jimmy Eat World', ...",31,17866039,44,Songs I used to listen to
159,DayDreams,false,64159,1509321600,22,18,2,"[{'pos': 0, 'artist_name': 'Lana Del Rey', 'tr...",12,4245112,18,"&quot;Here&#x27;s to the ones who dream, fooli..."


In [31]:
playlists_data.shape

(20000, 12)

In [36]:
##################################################################################
# REMEMBER TO DELETE THIS CELL - deprecated #
##################################################################################

data_dir = 'spotify-million-playlist/data'
output_file = 'working/playlists.csv'

with open(output_file, 'w') as f:
    first_file = True
    for file in tqdm(os.listdir(data_dir), desc="Processing playlists"):
        if file.endswith('.json'):
            file_path = os.path.join(data_dir, file)
            with open(file_path, 'r') as json_file:
                data = json.load(json_file)
                playlist_df = pd.DataFrame(data['playlists'])
                playlist_df.to_csv(f, header=first_file, index=False)
                if first_file:
                    first_file = False
print("CSV file successfully written!")

Processing playlists: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [17:33<00:00,  1.05s/it]

CSV file successfully written!


To conserve HDD space, we proceede to delete the original Spotify-Million-Playlist data and any other files that are no longer needed.

### 2. Extracting Audio Features using Spotipy Library

This code is designed to extract audio features for each track in the Spotify Million Playlist Dataset. The audio features are extracted using the Spotipy library which allows access to the Spotify API.

These audio features include: danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_ms, and time_signature. A description of each audio feature can be found here: [Spotify for Developers.](https://developer.spotify.com/documentation/web-api/reference/get-audio-features)

(By extracting the audio features, we gain insights into the characteristics of each track and can use this information to analyze the playlist as a whole. This step is essential for any analysis that requires an understanding of the musical properties of each track.)

In [38]:
with open('working/unique_tracks.csv', 'r') as f:
    reader = csv.reader(f)
    unique_tracks = list(reader)[0]

In [39]:
print('No. of unique Tracks: ', len(unique_tracks))

No. of unique Tracks:  2262292


In [49]:
# Initialize Spotify API
load_dotenv('spotify_credentials.env')
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(), requests_timeout=40)

In [54]:
#Fetching the spotify audio features, in chunks of 100 uri at a time.
start = 0
end = len(unique_tracks)

# Initialize empty list to hold audio features
itemlist = []

with tqdm(total=end) as pbar:
    for i in range(start, end, 100):
        temp_list = unique_tracks[i:i+100]
        try:
            playlists = sp.audio_features(temp_list)
            # Adding audio feature to itemlist
            itemlist.extend(playlists)
        except Exception as error:
            print('Error:', error)
            time.sleep(3)
            continue
        pbar.update(len(temp_list))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2262292/2262292 [1:05:38<00:00, 574.37it/s]


In [ ]:
# Initialize an empty list
after_audio_features_list = []

# Iterate over each item
for item in itemlist:
    # Check if the item is not None
    if item is not None:
        # Append the item to the list
        after_audio_features_list.append(item)

# Create DataFrame
after_audio_features = pd.DataFrame(after_audio_features_list)

# Save as csv
after_audio_features.to_csv('working/song_audio_features.csv', index= False)

A sample of the Audio Features extracted.

In [6]:
after_audio_features = pd.read_csv('working/song_audio_features.csv')
after_audio_features.head(3)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.648,0.223,7,-18.995,1,0.0305,0.7110,0.000033,0.360,0.490,115.443,audio_features,53oYDp9wNqCWbaa3FrNLoW,spotify:track:53oYDp9wNqCWbaa3FrNLoW,https://api.spotify.com/v1/tracks/53oYDp9wNqCW...,https://api.spotify.com/v1/audio-analysis/53oY...,344027,4
1,0.563,0.795,9,-4.904,1,0.1030,0.1700,0.000000,0.132,0.566,91.470,audio_features,4JYbUk8tzGLr5UEr7UP6F8,spotify:track:4JYbUk8tzGLr5UEr7UP6F8,https://api.spotify.com/v1/tracks/4JYbUk8tzGLr...,https://api.spotify.com/v1/audio-analysis/4JYb...,1026116,4
2,0.525,0.923,1,-3.311,0,0.0890,0.0392,0.000000,0.189,0.531,148.420,audio_features,7f0fbvmmNyQxCldwORvPUg,spotify:track:7f0fbvmmNyQxCldwORvPUg,https://api.spotify.com/v1/tracks/7f0fbvmmNyQx...,https://api.spotify.com/v1/audio-analysis/7f0f...,230173,4


Dropping unwanted columns.

In [7]:
after_audio_features = after_audio_features.drop(['type', 'id', 'track_href', 'analysis_url'], axis=1)
after_audio_features.to_csv('working/song_audio_features.csv', index= False)
after_audio_features.head(3)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,time_signature
0,0.648,0.223,7,-18.995,1,0.0305,0.7110,0.000033,0.360,0.490,115.443,spotify:track:53oYDp9wNqCWbaa3FrNLoW,344027,4
1,0.563,0.795,9,-4.904,1,0.1030,0.1700,0.000000,0.132,0.566,91.470,spotify:track:4JYbUk8tzGLr5UEr7UP6F8,1026116,4
2,0.525,0.923,1,-3.311,0,0.0890,0.0392,0.000000,0.189,0.531,148.420,spotify:track:7f0fbvmmNyQxCldwORvPUg,230173,4


#### 2.1. Extracting Adiccional Track Info using Spotipy Library

We attempted to extract additional track information, such as track popularity, the presence of explicit content, and the album genres of the artist or album.

Unfortunately, due to the constraints of the Spotify API, we were only able to apply the code to a sizable portion of the dataset. Furthermore, we encountered other issues such as inconsistencies in the API's response structure and a lack of these features in a large portion of the dataset.

As a result, we had to use workarounds that resulted in messy code, and we ultimately had to abandon the idea of incorporating some of the above-mentioned features.


In [5]:
# Initialize Spotify API
load_dotenv('spotify_credentials.env')
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(), requests_timeout=40)

In [3]:
# This cell should be executed after creating the dataset ("playlists_ex_data") in section 3.

playlists_ex_data = pd.read_csv('working/playlists_ex_data.csv')

track_list = playlists_ex_data['track_uri'].unique().tolist()
track_list = list(map(lambda x: x.replace('spotify:track:', ''), track_list))

We decided to separate the track list into other lists extracted from the API due to the API's limitations and the lack of some features. We thought that using sp.albums and sp.artists instead of sp.tracks may yield better results.

In [6]:
# using track_list 

#Fetching the spotify audio features, in chunks of 100 uri at a time.
utrack_list = list(set(track_list))
start = 0
end = len(track_list)

# Initialize empty list to hold audio features
album_release_date_t = []
album_id_t = []

artist_id_t = []

track_popularity_t = []
track_explicit_t = []

with tqdm(total=end) as pbar:
    for i in range(start, end, 50):
        temp_list = track_list[i:i+50]
        try:
            extra_features = sp.tracks(temp_list)['tracks']
            for x in range(len(extra_features)):
                if extra_features[x] is not None:
                    # Adding to the lists
                    album_release_date_t.append(extra_features[x]['album']['release_date'])
                    album_id_t.append(extra_features[x]['album']['id'])

                    artist_id_t.append(extra_features[x]['artists'][0]['id'])

                    track_popularity_t.append(extra_features[x]['popularity'])
                    track_explicit_t.append(extra_features[x]['explicit'])
                else:
                    album_release_date_t.append(None)
                    album_id_t.append(None)
                    artist_id_t.append(None)
                    track_popularity_t.append(None)
                    track_explicit_t.append(None)
        except Exception as error:
            print('Error:', error)
            print(i)
            time.sleep(3)
            continue
        pbar.update(len(temp_list))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268230/268230 [16:13<00:00, 275.52it/s]


In [11]:
# SAVING lists to working environment

with open("working/album_release_date_t", "wb") as fp:
    pickle.dump(album_release_date_t, fp)
    
with open("working/album_id_t", "wb") as fp:
    pickle.dump(album_id_t, fp)
    
with open("working/artist_id_t", "wb") as fp:
    pickle.dump(artist_id_t, fp)
    
with open("working/track_popularity_t", "wb") as fp:
    pickle.dump(track_popularity_t, fp)
    
with open("working/track_explicit_t", "wb") as fp:
    pickle.dump(track_explicit_t, fp)

In [5]:
# LOADING lists

with open("working/album_release_date_t", "rb") as fp:
    album_release_date_t = pickle.load(fp)
    
with open("working/album_id_t", "rb") as fp:
    album_id_t = pickle.load(fp)
    
with open("working/artist_id_t", "rb") as fp:
    artist_id_t = pickle.load(fp)
    
with open("working/track_popularity_t", "rb") as fp:
    track_popularity_t = pickle.load(fp)
    
with open("working/track_explicit_t", "rb") as fp:
    track_explicit_t = pickle.load(fp)

##### 2.1.1 Attempting to Extract Other Track Details

Although the code below functions correctly, it was not utilized due to the aforementioned issues.

In [5]:
album_id_t_1 = album_id_t[:154000]
album_id_t_2 = album_id_t[154000:]

In [ ]:
# Extracting track info 
# Using ALBUMS ID (PART 1 of 2)

end = len(prova)

# Initialize empty list to hold audio features
album_genres_1 = []
album_popularity = []

with tqdm(total=end) as pbar:
    for i in range(0, end, 20):
        temp_list = prova[i:i+20]
        try:
            extra_features = sp.albums(temp_list)['albums']
            for x in range(len(extra_features)):
                if extra_features[x] is not None:
                    # Adding audio info to lists
                    album_genres_1.append(extra_features[x]['genres'])
                    album_popularity.append(extra_features[x]['popularity'])
                else:
                    album_genres_1.append('None')
                    album_popularity.append('None')
        except Exception as error:
            print('Error:', error)
            time.sleep(3)
            continue
        pbar.update(len(temp_list))

As shown in the following printout, it returns a higher number of 'None' values than anticipated.

In [102]:
print(album_genres_1)
print(album_popularity)

[[], None, None]
[65, None, None]


In [120]:
end = len(prova)

# Initialize empty list to hold audio features
extra_features = []

with tqdm(total=end) as pbar:
    for i in range(0, end, 20):
        temp_list = prova[i:i+20]
        try:
            extra_features = sp.albums(temp_list)['albums']

            extra_features.append(extra_features)

        except:
            for item in temp_list: # add None values to the lists
                album_genres_1.append(None)
                album_popularity.append(None)
        pbar.update(len(temp_list))


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16.86it/s]


We tested the information returned from the Spotify API to determine whether there were any issues with the data format. However, as shown in the following printout, it returned null values.

Although we had already retrieved all of the album track details, we assumed that the issue was with our code and therefore decided to scrap that portion of the code and start anew.

In [119]:
extra_features

[[...]]

In [ ]:
# Using ARTISTS ID

# Initialize empty list to hold audio features
artist_genres = []
artist_popularity = []

with tqdm(total=end) as pbar:
    for i in range(0, len(artist_id_t), 50):
        temp_list = artist_id_t[i:i+50]
        try:
            extra_features = sp.artists(temp_list)['artists']
            for x in range(len(extra_features)):
                # Adding audio info to lists
                artist_genres.append(extra_features[x]['genres'])
                artist_popularity.append(extra_features[x]['popularity'])
        except:
            for _ in range(len(temp_list)): # add None values to the lists
                artist_genres.append(None)
                artist_popularity.append(None)
            continue
        pbar.update(len(temp_list))

In [6]:
data_info_to_dic = {
    'album_release_date': album_release_date_t,
    'track_popularity': track_popularity_t,
    'track_explicit': track_explicit_t,
    'track_id': track_list}

df_track_info = pd.DataFrame(data_info_to_dic)

### 3. Adding Audio Features to Playlist Dataset and Data Cleaning

This code extracts track values from the track column in the Playlist dataset, and merges the audio features data from a separate dataframe using the track_uri column. The resulting merged dataset is then cleaned to remove any irrelevant or missing data.

The final dataset contains 20,000 playlists with over 1.33 million songs, providing a comprehensive source of audio feature data for analysis and modeling.

In [2]:
after_audio_features = pd.read_csv('working/song_audio_features.csv')
playlists_data = pd.read_csv('working/playlists_data.csv')

In [ ]:
playlists_data = playlists_data.explode('tracks')

playlists_data['track_uri'] = playlists_data['tracks'].apply(lambda x: x['track_uri'])
playlists_data['pos'] = playlists_data['tracks'].apply(lambda x: x['pos'])
playlists_data['artist_name'] = playlists_data['tracks'].apply(lambda x: x['artist_name'])
playlists_data['track_name'] = playlists_data['tracks'].apply(lambda x: x['track_name'])
playlists_data['duration_ms'] = playlists_data['tracks'].apply(lambda x: x['duration_ms'])
playlists_data['album_name'] = playlists_data['tracks'].apply(lambda x: x['album_name'])

In [36]:
playlists_data = playlists_data.drop(['tracks'], axis=1)

In [38]:
playlists_data.shape

(1330043, 16)

In [48]:
playlists_ex_data = pd.merge(playlists_data, after_audio_features, left_on='track_uri', right_on='uri', how='inner')
playlists_ex_data = playlists_ex_data.sort_values(['pid', 'pos'])

In [32]:
playlists_ex_data = playlists_ex_data.drop(['uri', 'duration_ms_x'], axis=1)
playlists_ex_data = playlists_ex_data.reindex(columns=['pid', 'pos', 'name', 'description', 'modified_at', 'num_tracks', 'num_albums', 'num_followers', 'num_edits', 'artist_name', 'track_name', 'album_name', 'track_uri','danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms_y', 'time_signature'])
playlists_ex_data = playlists_ex_data.rename(columns={'duration_ms_y': 'duration_ms'})

In [35]:
playlists_ex_data.to_csv('working/playlists_ex_data.csv', index= False)

In [36]:
playlists_ex_data.shape

(1330040, 26)

A sample of the dataset:

In [38]:
playlists_ex_data[playlists_ex_data['description'].notna()].head(3)

,pid,pos,name,description,modified_at,num_tracks,num_albums,num_followers,num_edits,artist_name,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
330,11007,0,the playlist,this playlist contains pretty much all my favo...,1508803200,169,150,2,131,Zedd,...,-3.480,1,0.0720,0.0398,0.000000,0.0749,0.176,128.000,271427,4
331,11007,1,the playlist,this playlist contains pretty much all my favo...,1508803200,169,150,2,131,OutKast,...,-2.261,0,0.0664,0.1030,0.000532,0.1740,0.965,79.526,235213,4
332,11007,2,the playlist,this playlist contains pretty much all my favo...,1508803200,169,150,2,131,Foster The People,...,-5.849,0,0.0292,0.1450,0.115000,0.0956,0.965,127.975,239600,4


In [39]:
playlists_ex_data.columns

Index(['pid', 'pos', 'name', 'description', 'modified_at', 'num_tracks',
       'num_albums', 'num_followers', 'num_edits', 'artist_name', 'track_name',
       'album_name', 'track_uri', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature'],
      dtype='object')

#### 3.1 Adding addicional track info to the Playlist Dataset

We add the album release date, the track popularity, and the presence of explicit content to the Dataset.

In [10]:
playlists_ex_data = pd.read_csv('working/playlists_ex_data.csv')

In [14]:
playlists_ex_data['track_uri'] = playlists_ex_data['track_uri'].apply(lambda x: x.replace('spotify:track:', ''))
playlists_dataset = pd.merge(playlists_ex_data, df_track_info, left_on='track_uri', right_on='track_id', how='left')

In [36]:
playlists_dataset = playlists_dataset.drop(['track_id'], axis=1)

In [41]:
playlists_dataset.to_csv('working/playlists_dataset.csv', index= False)

In [37]:
playlists_dataset.shape

(1330040, 29)

In [39]:
playlists_dataset.columns

Index(['pid', 'pos', 'name', 'description', 'modified_at', 'num_tracks',
       'num_albums', 'num_followers', 'num_edits', 'artist_name', 'track_name',
       'album_name', 'track_uri', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature',
       'album_release_date', 'track_popularity', 'track_explicit'],
      dtype='object')

In [40]:
playlists_dataset[playlists_dataset.notna().all(axis=1)].head(3)

,pid,pos,name,description,modified_at,num_tracks,num_albums,num_followers,num_edits,artist_name,...,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,album_release_date,track_popularity,track_explicit
330,11007,0,the playlist,this playlist contains pretty much all my favo...,1508803200,169,150,2,131,Zedd,...,0.0398,0.000000,0.0749,0.176,128.000,271427,4,2012-01-01,76.0,False
331,11007,1,the playlist,this playlist contains pretty much all my favo...,1508803200,169,150,2,131,OutKast,...,0.1030,0.000532,0.1740,0.965,79.526,235213,4,2003,83.0,False
332,11007,2,the playlist,this playlist contains pretty much all my favo...,1508803200,169,150,2,131,Foster The People,...,0.1450,0.115000,0.0956,0.965,127.975,239600,4,2011-05-23,84.0,False


In [19]:
experiment.end()

emissions_tracker.stop()

[codecarbon WARNING @ 12:21:38] Background scheduler didn't run for a long period (81s), results might be inaccurate
[codecarbon INFO @ 12:21:38] Energy consumed for RAM : 0.000452 kWh. RAM Power : 11.759862899780273 W
[codecarbon INFO @ 12:21:38] Energy consumed for all CPUs : 0.001634 kWh. All CPUs Power : 42.5 W
[codecarbon INFO @ 12:21:38] 0.002087 kWh of electricity used since the begining.


0.00039647753147580814

[codecarbon INFO @ 12:21:38] Energy consumed for RAM : 0.004164 kWh. RAM Power : 11.759862899780273 W
[codecarbon INFO @ 12:21:38] Energy consumed for all CPUs : 0.015051 kWh. All CPUs Power : 42.5 W
[codecarbon INFO @ 12:21:38] 0.019214 kWh of electricity used since the begining.
[codecarbon INFO @ 12:21:53] Energy consumed for RAM : 0.004213 kWh. RAM Power : 11.759862899780273 W
[codecarbon INFO @ 12:21:53] Energy consumed for all CPUs : 0.015228 kWh. All CPUs Power : 42.5 W
[codecarbon INFO @ 12:21:53] 0.019441 kWh of electricity used since the begining.
[codecarbon INFO @ 12:22:08] Energy consumed for RAM : 0.004262 kWh. RAM Power : 11.759862899780273 W
[codecarbon INFO @ 12:22:08] Energy consumed for all CPUs : 0.015405 kWh. All CPUs Power : 42.5 W
[codecarbon INFO @ 12:22:08] 0.019667 kWh of electricity used since the begining.
